# MiniProyecto 4: Optimización de parámetros

¡Bienvenidos al cuarto (mini)proyecto de la carrera de Data Science de Acamica! 

En este proyecto vamos a seguir trabajando (por última vez) con el dataset de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar). El objetivo en este caso es optimizar los parámetros de los algoritmos que usamos en el proyecto pasado.

El dataset es el mismo del proyecto 3. Recordemos que las columnas que se agregan son:

* `barrios_match`: si coincide el barrio publicado con el geográfico vale 1, si no 0.

* `PH`, `apartment`, `house`: variables binarias que indican el tipo de propiedad.

* dummies de barrios: variables binarias con 1 o 0 según el barrio.

La métrica que vamos a usar para medir es RMSE (raíz del error cuadréatico medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

## Pandas - Levantamos el dataset

In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = 'dataset/datos_properati_limpios_model.csv'
df = pd.read_csv(path_dataset)
df.head()

,lat,lon,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,rooms,barrio_match,PH,apartment,house,...,VILLA LUGANO,VILLA LURO,VILLA ORTUZAR,VILLA PUEYRREDON,VILLA REAL,VILLA RIACHUELO,VILLA SANTA RITA,VILLA SOLDATI,VILLA URQUIZA,outlier_price_m2
0,-34.589,-58.417,170000.000,40.000,38.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-34.591,-58.418,90000.000,27.000,27.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,-34.587,-58.437,150000.000,44.000,44.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,-34.593,-58.428,154000.000,58.000,58.000,2,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-34.593,-58.428,154000.000,58.000,58.000,3,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


**Separá** el dataset en entrenamiento (80%) y test (20%) utilizando como target la columna `price_aprox_usd`

In [2]:
# Esto es lo que hace
import numpy as np
np.random.seed(123)
from sklearn.model_selection import train_test_split
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print(X_train.shape[0], X_test.shape[0])

5100 1276


## Scikit-learn - Entrenamiento

Para repasar los parámetros de árboles de decisión en Scikit-learn: 

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

En primer lugar veamos como hacer cross validation. Para eso necesitamos definir la cantidad de folds, en este caso vamos a usar 5.

GridSearchCV nos permite testear a través de un espacio de búsqueda de parámetros la mejor combinación posible dado un estimador.

Por ejemplo, en este caso probamos la profundidad máxima y la máxima cantidad de features para hacer los split. Ambos entre 1 y 5.
Recordemos que para hacer la optimización scikit-learn usa la métrica `neg_mean_squared_error` en lugar de `mean_squared_error`.

**Creá** una variable `param_grid` con valores del 1 al 5 para los atributos `max_depth` y `max_features`. 

In [3]:
param_grid = {'max_depth' : np.arange(1,6,1),
             'max_features': np.arange(1,6,1),
             'random_state':[0]}

**Importá** `GridSearchCV` y `DecisionTreeRegressor`.

**Creá** una variable `grid_search` y asignale un `GridSearchCV` que recorra el `param_grid` que creaste con el algoritmos `DecisionTreeRegressor` y el un scoring de `neg_mean_squared_error`

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
reg = DecisionTreeRegressor()

A continuación, realizá el `fit` del `grid_search` con el conjunto de entrenamiento

In [5]:
model = GridSearchCV(reg, param_grid=param_grid,scoring='neg_mean_squared_error', cv=5)
model.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': array([1, 2, 3, 4, 5]),
                         'max_features': array([1, 2, 3, 4, 5]),
                         'ra

Revisemos los resultados. Recordemos que no están expresados en RMSE.

In [6]:
print('Mean Test Scores:')
print(model.cv_results_['mean_test_score'])
print('-----------------')
print('Std Test Scores:')
print(model.cv_results_['std_test_score'])

Mean Test Scores:
[-9.79790172e+08 -9.79795656e+08 -9.67431346e+08 -9.67431346e+08
 -9.67431346e+08 -9.78942683e+08 -9.76100752e+08 -9.08413692e+08
 -9.66098817e+08 -9.65565625e+08 -9.77325878e+08 -9.52598423e+08
 -7.29969370e+08 -8.93010237e+08 -9.57268503e+08 -9.69801712e+08
 -9.38732015e+08 -7.65093675e+08 -8.63886725e+08 -8.77792421e+08
 -9.28570183e+08 -7.74789804e+08 -7.60342205e+08 -8.61045681e+08
 -9.16444472e+08]
-----------------
Std Test Scores:
[2.55070797e+07 2.56483807e+07 2.68426195e+07 2.68426195e+07
 2.68426195e+07 2.59726327e+07 2.38770090e+07 8.06224430e+07
 2.68539629e+07 2.99019010e+07 2.16013445e+07 2.24985450e+07
 1.73989925e+07 3.87337936e+07 2.89062027e+07 3.02074604e+07
 2.30421530e+07 1.04119001e+08 8.36515621e+07 1.04309903e+08
 6.27035872e+07 6.10006130e+07 8.88882602e+07 9.29480409e+07
 2.92191868e+07]


**Mostrá** los `grid_scores` obtenidos durante el `grid_search`

In [7]:
scores = pd.DataFrame(model.cv_results_)
scores.head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007,0.003,0.001,0.000,1,1,0,"{'max_depth': 1, 'max_features': 1, 'random_st...",-997527974.201,-992475505.903,-934306596.646,-1004573568.526,-970067215.408,-979790172.137,25507079.717,24
1,0.004,0.000,0.001,0.000,1,2,0,"{'max_depth': 1, 'max_features': 2, 'random_st...",-997975870.246,-992475505.903,-934202375.860,-1004635347.873,-969689178.103,-979795655.597,25648380.710,25
2,0.004,0.000,0.001,0.000,1,3,0,"{'max_depth': 1, 'max_features': 3, 'random_st...",-986068693.940,-972814621.276,-920043980.268,-997926096.305,-960303338.034,-967431345.965,26842619.528,17
3,0.004,0.000,0.001,0.000,1,4,0,"{'max_depth': 1, 'max_features': 4, 'random_st...",-986068693.940,-972814621.276,-920043980.268,-997926096.305,-960303338.034,-967431345.965,26842619.528,17
4,0.004,0.000,0.001,0.000,1,5,0,"{'max_depth': 1, 'max_features': 5, 'random_st...",-986068693.940,-972814621.276,-920043980.268,-997926096.305,-960303338.034,-967431345.965,26842619.528,17


De esta manera, el valor con mejor resultado (dado el espacio de búsqueda definido) es `max_depth` 3 y `max_features` 3.

**Mostrá** el mejor score y los mejores parámetros encontrados por `grid_search`

In [8]:
print("Mejores parametros: "+str(model.best_params_))
print("Mejor Score: "+str(model.best_score_)+'\n')

Mejores parametros: {'max_depth': 3, 'max_features': 3, 'random_state': 0}
Mejor Score: -729969370.3438246



Convertimos a RMSE.

In [9]:
def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)

In [10]:
nmsq2rmse(model.best_score_)

27017.95

__Encontrar el mejor modelo para el espacio de búsqueda dado__

* `"min_samples_split": [2, 10, 20]`
* `"max_depth": [None, 2, 5, 10, 15]`
* `"min_samples_leaf": [1, 5, 10, 15]`
* `"max_leaf_nodes": [None, 5, 10, 20]`

Recordemos que `GridSearchCV` tiene como parámetro default `refit=True`. Esto significa que luego de hacer la corrida se ajusta el mejor modelo al conjunto de datos de entrada. De esta manera, se puede predecir directamente usando `best_estimator_`.

In [11]:
param_grid= {"min_samples_split": [2, 10, 20],
            "max_depth": [None, 2, 5, 10, 15],
            "min_samples_leaf": [1, 5, 10, 15],
            "max_leaf_nodes": [None, 5, 10, 20]}

model_1 = GridSearchCV(reg, param_grid=param_grid,scoring='neg_mean_squared_error',refit=True, cv=5)
model_1.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [None, 2, 5, 10, 15],
                         'max_leaf_nodes': [None, 5, 10, 20],
                         'min_sam

In [12]:
optimised_decision_tree = model_1.best_estimator_
optimised_decision_tree

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=10,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=15, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

__Evaluemos en testing el desempeño de este modelo.__

Como venimos trabajando, el resultado en testing será la medición que usaremos como benchmark para comparar este modelos con otros en el futuro, puesto que no estuvo en contacto con el dataset de test para la calibración.

In [13]:
from sklearn.metrics import mean_squared_error
y_opt_pred = optimised_decision_tree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

21301.0

Vemos los primeros 10 resultados de la predicción del valor de propiedades.

In [14]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_opt_pred)

In [15]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [16]:
predicciones.head(10)

,Valor real,Valor Pred,Dif(+/-)
0,80000.000,103438.660,23438.660
1,128000.000,135705.882,7705.882
2,150000.000,156075.759,6075.759
3,85000.000,102400.991,17400.991
4,135000.000,135571.622,571.622
5,135000.000,109560.000,25440.000
6,68000.000,75181.250,7181.250
7,110000.000,140444.444,30444.444
8,134000.000,158431.250,24431.250
9,110000.000,76701.202,33298.798
